In [5]:
!pip install trafilatura

  Using cached babel-2.17.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 3.6 MB/s eta 0:00:00a 0:00:01
Using cached babel-2.17.0-py3-none-any.whl (10.2 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1
  Attempting uninstall: lxml
    Found existing installation: lxml 5.2.1
    Uninstalling lxml-5.2.1:
      Successfully uninstalled lxml-5.2.1
  Attempting uninstall: charset_normalizer
    Found existing installation: charset-normalizer 3.3.2
    Uninstalling charset-normalizer-3.3.2:
      Successfully uninstalled charset-normalizer-3.3.2
  Attempting uninstall: babel
    Found existing installation: Babel 2.11.0
    Uninstalling Babel-

In [17]:
import pandas as pd
from trafilatura import fetch_url, extract
import json

In [18]:
df_urls = pd.read_csv('/Users/caio.mello/Documents/NetzDG_project/NetzDG_data/philophia_perennis_scraping/philosophia_perennis_Gesetz+zur+Verbesserung+der+Rechtsdurchsetzung+in+sozialen+Netzwerken.csv')

In [19]:
df_urls.head()

,Unnamed: 0,URL
0,0,https://philosophia-perennis.com/2021/06/15/ze...
1,1,https://philosophia-perennis.com/2018/03/05/rt...
2,2,https://philosophia-perennis.com/2017/06/29/bu...


In [20]:
extracted_data_list = []
processed_urls = []
processed_urls_set = set(processed_urls)

In [21]:
progress_filename = 'extracted_data_progress.json'

for index, row in df_urls.iterrows():
    url = row['URL']

    if url in processed_urls_set:
        print(f"URL already processed, skipping: {url}")
        continue # Skip to the next URL

    print(f"Processing URL: {url}")
    try:
        downloaded_html = fetch_url(url)
        if downloaded_html:
            extracted_data = extract(downloaded_html, output_format="json", with_metadata=True)
            if extracted_data:
                extracted_data_list.append(extracted_data)
                # Save progress after each successful extraction
                try:
                    with open(progress_filename, 'w') as f:
                        json.dump(extracted_data_list, f, indent=4)
                    # Add the successfully processed URL to the set
                    processed_urls_set.add(url)
                    print(f"Progress saved to {progress_filename}")
                except IOError as e:
                    print(f"Error saving progress to {progress_filename}: {e}")
            else:
                print(f"Could not extract data from: {url}")
        else:
            print(f"Could not download content from: {url}")
    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")


Processing URL: https://philosophia-perennis.com/2021/06/15/zensur-jetzt-geht-es-telegram-an-den-kragen/
Progress saved to extracted_data_progress.json
Processing URL: https://philosophia-perennis.com/2018/03/05/rt-stahl/
Progress saved to extracted_data_progress.json
Processing URL: https://philosophia-perennis.com/2017/06/29/bundestag-netzwerkdurchsetzungsgesetz/
Progress saved to extracted_data_progress.json


In [22]:
# Convert the list of dictionaries into a pandas DataFrame
df_extracted = pd.DataFrame(extracted_data_list)

# Save the DataFrame to a CSV file without the index
output_filename = 'extracted_philosophia_perennis_data.csv'
df_extracted.to_csv(output_filename, index=False)

print(f"Extracted data saved to {output_filename}")

# Print the head and info of the created DataFrame
print("\nHead of the extracted data DataFrame:")
display(df_extracted.head())

print("\nInfo of the extracted data DataFrame:")
df_extracted.info()

Extracted data saved to extracted_philosophia_perennis_data.csv

Head of the extracted data DataFrame:


,0
0,"{""title"": ""Zensur: Jetzt geht es \""Telegram\"" ..."
1,"{""title"": ""Interview mit Rechtsanwalt Dr. Stah..."
2,"{""title"": ""Bundestag: Bekämpfung von News, die..."



Info of the extracted data DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      object
dtypes: object(1)
memory usage: 156.0+ bytes
